In [1]:
from textblob import TextBlob
import numpy as np
import pandas as pd
import nltk
pd.options.display.max_columns = None
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
plt.style.use('ggplot')
df = pd.read_csv('/Users/manu/Analisis Predictivo/Exámen 2/df.csv')
df.head(3)

#Divido entre val y train para probar los modelos.
train = df[df['review_scores_rating'].notnull()]
val = df[df['review_scores_rating'].isnull()]
val.reset_index(drop=True, inplace=True)


## SVR - Primera prueba

In [2]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from skopt import BayesSearchCV
from sklearn.preprocessing import MinMaxScaler

X = train.drop('review_scores_rating', axis=1)
y = train['review_scores_rating']

# Normalizar los datos de entrada
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Dividir el conjunto de datos normalizado en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2)

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'C': (0.001, 100.0, 'log-uniform'),
    'epsilon': (0.01, 1.0, 'uniform'),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': (0.001, 10.0, 'log-uniform'),
    'degree': (1, 5),
}

# Crear el modelo de SVR
model = SVR()

# Realizar la búsqueda bayesiana de hiperparámetros
opt = BayesSearchCV(model, param_space, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42)
opt.fit(X_train, y_train)

# Obtener el mejor modelo y los mejores parámetros
best_model = opt.best_estimator_
best_params = opt.best_params_

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.4f}".format(mse))

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coefficient of Determination (R^2): {:.2f}".format(r2))

# Imprimir los mejores parámetros encontrados
print("Best Parameters:")
for param, value in best_params.items():
    print("{}: {}".format(param, value))


Mean Squared Error: 0.0354
Coefficient of Determination (R^2): 0.47
Best Parameters:
C: 100.0
degree: 1
epsilon: 0.01
gamma: 0.001
kernel: sigmoid


In [9]:
# Predecir en la base 'val' sin la columna 'review_scores_rating'
X_val = val.drop('review_scores_rating', axis=1)
y_pred = best_model.predict(X_val)

# Crear un DataFrame con las predicciones
y_pred_df = pd.DataFrame(y_pred, columns=['review_scores_rating'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['id'], y_pred_df], axis=1)

# Imprimir el resultado
submission

/Users/manu/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


,id,review_scores_rating
0,568520730880917893,1.075546
1,568523400177884625,1.090044
2,568561397403760678,1.190415
3,568683128219348239,1.080894
4,570029413851129083,3.902695
...,...,...
1228,794322,1.067610
1229,800723,1.064613
1230,801544,1.070046
1231,802052,1.067747


In [23]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler

# Datos de entrenamiento
X_train = train.drop('review_scores_rating', axis=1)
y_train = train['review_scores_rating']

# Normalizar los datos de entrenamiento
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)

# Crear el modelo de SVR con los mejores parámetros
best_model = SVR(C=100.0, degree=1, epsilon=0.01, gamma=0.001, kernel='sigmoid')

# Ajustar el modelo con los datos de entrenamiento normalizados
best_model.fit(X_train_normalized, y_train)

# Predecir en la base de entrenamiento completa
y_pred_train = best_model.predict(X_train_normalized)

# Crear un DataFrame con las predicciones en la base de entrenamiento
y_pred_train_df = pd.DataFrame(y_pred_train, columns=['review_scores_rating'])

# Concatenar las predicciones con la columna 'id' de la base de entrenamiento
train_submission = pd.concat([train['id'], y_pred_train_df], axis=1)

# Imprimir el resultado en la base de entrenamiento
print("Predictions on training data:")
print(train_submission)

# Predecir en la base 'val' sin la columna 'review_scores_rating'
X_val = val.drop('review_scores_rating', axis=1)
X_val_normalized = scaler.transform(X_val)  # Normalizar los datos de validación

y_pred_val = best_model.predict(X_val_normalized)

# Crear un DataFrame con las predicciones en la base 'val'
y_pred_val_df = pd.DataFrame(y_pred_val, columns=['review_scores_rating'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['id'], y_pred_val_df], axis=1)

# Imprimir el resultado en la base 'val'
print("Predictions on validation data:")
print(submission)


Predictions on training data:
                      id  review_scores_rating
0                 528022              4.832904
1                   2818              4.898070
2                  20168              4.614304
3                  27886              4.911329
4                  28871              4.873147
...                  ...                   ...
4923  567407960487712181              5.010140
4924  567829002669570759              4.937266
4925  567927199868989828              4.988337
4926  567958632398468204              4.915264
4927  568514877303578111              5.024652

[4928 rows x 2 columns]
Predictions on validation data:
                      id  review_scores_rating
0     568520730880917893              5.026243
1     568523400177884625              4.946100
2     568561397403760678              4.239487
3     568683128219348239              5.006944
4     570029413851129083              4.924856
...                  ...                   ...
1228              79

In [17]:
import numpy as np

# Truncar las predicciones para asegurarse de que estén en el rango [0, 5]
y_pred_train_truncated = np.clip(y_pred_train, 0, 5)
y_pred_val_truncated = np.clip(y_pred_val, 0, 5)

# Crear un DataFrame con las predicciones truncadas en la base de entrenamiento
y_pred_train_truncated_df = pd.DataFrame(y_pred_train_truncated, columns=['review_scores_rating'])

# Concatenar las predicciones truncadas con la columna 'id' de la base de entrenamiento
train_submission_truncated = pd.concat([train['id'], y_pred_train_truncated_df], axis=1)

# Imprimir el resultado en la base de entrenamiento truncada
print("Truncated predictions on training data:")
print(train_submission_truncated)

# Crear un DataFrame con las predicciones truncadas en la base 'val'
y_pred_val_truncated_df = pd.DataFrame(y_pred_val_truncated, columns=['review_scores_rating'])

# Concatenar las predicciones truncadas con la columna 'id' de la base 'val'
submission_truncated = pd.concat([val['id'], y_pred_val_truncated_df], axis=1)

# Imprimir el resultado en la base 'val' truncada
print("Truncated predictions on validation data:")
print(submission_truncated)
submission_truncated.to_csv('submission.csv', index=False)

Truncated predictions on training data:
                      id  review_scores_rating
0                 528022              4.830216
1                   2818              4.899806
2                  20168              4.615985
3                  27886              4.912173
4                  28871              4.874291
...                  ...                   ...
4923  567407960487712181              5.000000
4924  567829002669570759              4.939771
4925  567927199868989828              4.988752
4926  567958632398468204              4.915091
4927  568514877303578111              5.000000

[4928 rows x 2 columns]
Truncated predictions on validation data:
                      id  review_scores_rating
0     568520730880917893              5.000000
1     568523400177884625              4.946852
2     568561397403760678              4.238538
3     568683128219348239              5.000000
4     570029413851129083              4.924846
...                  ...                   ...


## SVR - Segunda Prueba

In [37]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from skopt import BayesSearchCV
from sklearn.preprocessing import MinMaxScaler

X = train.drop('review_scores_rating', axis=1)
y = train['review_scores_rating']

#Normalizar los datos de entrada
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2)

param_space = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'epsilon': (0.01, 1.0),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': (0.001, 10.0),
    'degree': (1, 5),
}
model = SVR()

#Realizar la búsqueda bayesiana de hiperparámetros y ajustar el mejor modelo automáticamente
opt = BayesSearchCV(model, param_space, n_iter=50, cv=5, scoring='neg_mean_squared_error', refit=True)
opt.fit(X_train, y_train)

#Obtener el mejor modelo y los mejores parámetros
best_model = opt.best_estimator_
best_params = opt.best_params_

#Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.4f}".format(mse))

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coefficient of Determination (R^2): {:.2f}".format(r2))

print("Best Parameters:")
for param, value in best_params.items():
    print("{}: {}".format(param, value))

Mean Squared Error: 0.0104
Coefficient of Determination (R^2): 0.78
Best Parameters:
C: 0.1
degree: 4
epsilon: 0.11537046867819958
gamma: 2.511158244626176
kernel: linear


In [30]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler

# Datos de entrenamiento
X_train = train.drop('review_scores_rating', axis=1)
y_train = train['review_scores_rating']

scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)

best_model = SVR(C=0.1, degree=5, epsilon=0.017227358801368102, gamma=10.0, kernel='linear')

best_model.fit(X_train_normalized, y_train)

y_pred_train = best_model.predict(X_train_normalized)

y_pred_train_df = pd.DataFrame(y_pred_train, columns=['review_scores_rating'])

train_submission = pd.concat([train['id'], y_pred_train_df], axis=1)

print("Predictions on training data:")
print(train_submission)

# Predecir en la base 'val' sin la columna 'review_scores_rating'
X_val = val.drop('review_scores_rating', axis=1)
X_val_normalized = scaler.transform(X_val)  # Normalizar los datos de validación

y_pred_val = best_model.predict(X_val_normalized)

# Crear un DataFrame con las predicciones en la base 'val'
y_pred_val_df = pd.DataFrame(y_pred_val, columns=['review_scores_rating'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['id'], y_pred_val_df], axis=1)

# Imprimir el resultado en la base 'val'
print("Predictions on validation data:")
print(submission)


Predictions on training data:
                      id  review_scores_rating
0                 528022              4.830916
1                   2818              4.899134
2                  20168              4.617298
3                  27886              4.911853
4                  28871              4.871877
...                  ...                   ...
4923  567407960487712181              5.015138
4924  567829002669570759              4.941917
4925  567927199868989828              4.992021
4926  567958632398468204              4.916564
4927  568514877303578111              5.029367

[4928 rows x 2 columns]
Predictions on validation data:
                      id  review_scores_rating
0     568520730880917893              5.033844
1     568523400177884625              4.949070
2     568561397403760678              4.238696
3     568683128219348239              5.011970
4     570029413851129083              4.928847
...                  ...                   ...
1228              79

In [31]:
import numpy as np

# Truncar las predicciones para asegurarse de que estén en el rango [0, 5]
y_pred_train_truncated = np.clip(y_pred_train, 0, 5)
y_pred_val_truncated = np.clip(y_pred_val, 0, 5)

# Crear un DataFrame con las predicciones truncadas en la base de entrenamiento
y_pred_train_truncated_df = pd.DataFrame(y_pred_train_truncated, columns=['review_scores_rating'])

# Concatenar las predicciones truncadas con la columna 'id' de la base de entrenamiento
train_submission_truncated = pd.concat([train['id'], y_pred_train_truncated_df], axis=1)

# Imprimir el resultado en la base de entrenamiento truncada
print("Truncated predictions on training data:")
print(train_submission_truncated)

# Crear un DataFrame con las predicciones truncadas en la base 'val'
y_pred_val_truncated_df = pd.DataFrame(y_pred_val_truncated, columns=['review_scores_rating'])

# Concatenar las predicciones truncadas con la columna 'id' de la base 'val'
submission_truncated = pd.concat([val['id'], y_pred_val_truncated_df], axis=1)

# Imprimir el resultado en la base 'val' truncada
print("Truncated predictions on validation data:")
print(submission_truncated)


Truncated predictions on training data:
                      id  review_scores_rating
0                 528022              4.830916
1                   2818              4.899134
2                  20168              4.617298
3                  27886              4.911853
4                  28871              4.871877
...                  ...                   ...
4923  567407960487712181              5.000000
4924  567829002669570759              4.941917
4925  567927199868989828              4.992021
4926  567958632398468204              4.916564
4927  568514877303578111              5.000000

[4928 rows x 2 columns]
Truncated predictions on validation data:
                      id  review_scores_rating
0     568520730880917893              5.000000
1     568523400177884625              4.949070
2     568561397403760678              4.238696
3     568683128219348239              5.000000
4     570029413851129083              4.928847
...                  ...                   ...


In [32]:
submission_truncated.to_csv('submission.csv', index=False)